In [13]:
import pandas as pd
import os
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import pickle
import networkx as nx

In [24]:
top20 = {}
directory = 'GWAS_result'
for filename in os.listdir(directory):
    if filename.endswith('.tsv'):
        group_num = int(filename.split('_')[0][5:])
        top20[group_num] = pd.read_csv(os.path.join(directory, filename), sep = '\t').feature.value_counts().keys().tolist()[:20]

In [15]:
from collections import Counter
from typing import List
import obonet

# load the HPO ontology graph
graph = obonet.read_obo('/home/lucytian/code/hp.obo')

def find_hpo_ids(strings: List[str]) -> List[str]:
    # initialize the set of HPO IDs to an empty set
    hpo_ids = set()
    
    # iterate over each string
    for string in strings:
        # split the string into words using the underscore character
        words = string.split('_')
        
        # initialize a counter to keep track of the number of matching words for each node
        node_counters = Counter()
        
        # iterate over each word and increment the counter for each node that matches
        for word in words:
            for node in graph.nodes():
                if graph.nodes[node]['name'].lower().find(word.lower()) != -1:
                    node_counters[node] += 1
        
        # filter the nodes based on the number of matching words, keeping only the nodes that have at least three matching words
        matching_nodes = [node for node in node_counters if node_counters[node] >= 3]
        
        # add the matching nodes to the set of HPO IDs
        hpo_ids |= set(matching_nodes)
    
    # return the list of matching HPO IDs
    return list(hpo_ids)

In [27]:
sorted_top20 = dict(sorted(top20.items()))
sorted_top20

{1: ['Free_Cholesterol_to_Total_Lipids_in_Very_Large_VLDL_percentage_(23587-0.0)',
  'Cholesteryl_Esters_to_Total_Lipids_in_Large_VLDL_percentage_(23591-0.0)',
  'Concentration_of_Small_HDL_Particles_(23572-0.0)',
  'Cholesterol_in_Small_HDL_(23575-0.0)',
  'Total_Lipids_in_Small_HDL_(23573-0.0)',
  'Phospholipids_to_Total_Lipids_in_Chylomicrons_and_Extremely_Large_VLDL_percentage_(23579-0.0)',
  'Triglycerides_to_Total_Lipids_in_Very_Large_VLDL_percentage_(23588-0.0)',
  'Free_Cholesterol_in_Small_HDL_(23577-0.0)',
  'Phospholipids_in_Small_HDL_(23574-0.0)',
  'Free_Cholesterol_in_Very_Large_HDL_(23556-0.0)',
  'Average_Diameter_for_VLDL_Particles_(23431-0.0)',
  'Total_Lipids_in_Medium_LDL_(23538-0.0)',
  'Cholesteryl_Esters_in_Medium_LDL_(23541-0.0)',
  'Total_Lipids_in_Lipoprotein_Particles_(23423-0.0)',
  'Polyunsaturated_Fatty_Acids_to_Monounsaturated_Fatty_Acids_ratio_(23458-0.0)',
  'Cholesterol_to_Total_Lipids_in_Large_VLDL_percentage_(23590-0.0)',
  'Total_Phospholipids_in_Li

In [29]:
hpos = []
for i in top20.values():
    hpos.append(find_hpo_ids(i))

In [30]:
[len(hpo) for hpo in hpos]

[36, 37, 44, 37, 45, 36, 45, 36, 44, 44, 37, 44, 44, 44, 44]

In [34]:
import json

df = pd.read_json('/home/lucytian/data/shepherd_data/patients/simulated_patients/patients.jsonl', 'r', lines=True)
df

<ipython-input-34-f73b90cd04ba>:3: FutureWarning: Starting with Pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only
  df = pd.read_json('/home/lucytian/data/shepherd_data/patients/simulated_patients/patients.jsonl', 'r', lines=True)


,id,positive_phenotypes,all_candidate_genes
0,Group_1,"[HP:0009101, HP:0005274, HP:0010750, HP:000015...","[ENSG00000281919, ENSG00000273936, ENSG0000028..."
1,Group_2,"[HP:0002922, HP:0010750, HP:0031956, HP:000082...","[ENSG00000261992, ENSG00000277909, ENSG0000028..."
2,Group_3,"[HP:0030863, HP:0002922, HP:0000234, HP:003195...","[ENSG00000198888, ENSG00000198899, ENSG0000019..."
3,Group_4,"[HP:0007517, HP:0003731, HP:0031956, HP:000082...","[ENSG00000279566, ENSG00000281775, ENSG0000027..."
4,Group_5,"[HP:0002922, HP:0000234, HP:0000151, HP:003195...","[ENSG00000198712, ENSG00000198840, ENSG0000021..."
5,Group_6,"[HP:0002922, HP:0000234, HP:0000151, HP:000082...","[ENSG00000198888, ENSG00000198899, ENSG0000021..."
6,Group_7,"[HP:0030500, HP:0002922, HP:0005274, HP:001075...","[ENSG00000198888, ENSG00000198899, ENSG0000019..."
7,Group_8,"[HP:0000151, HP:0000822, HP:0001382, HP:000055...","[ENSG00000274746, ENSG00000231543, ENSG0000020..."
8,Group_9,"[HP:0002922, HP:0005274, HP:0003731, HP:000015...","[ENSG00000280618, ENSG00000277421, ENSG0000027..."
9,Group_10,"[HP:0003731, HP:0005274, HP:0001382, HP:000082...","[ENSG00000277101, ENSG00000275482, ENSG0000027..."


In [36]:
updated = []
phenotypes = df.positive_phenotypes.tolist()
for i in np.arange(15):
    updated.append(phenotypes[i] + hpos[i])

In [37]:
df['updated_phenotypes'] = updated
df

,id,positive_phenotypes,all_candidate_genes,updated_phenotypes
0,Group_1,"[HP:0009101, HP:0005274, HP:0010750, HP:000015...","[ENSG00000281919, ENSG00000273936, ENSG0000028...","[HP:0009101, HP:0005274, HP:0010750, HP:000015..."
1,Group_2,"[HP:0002922, HP:0010750, HP:0031956, HP:000082...","[ENSG00000261992, ENSG00000277909, ENSG0000028...","[HP:0002922, HP:0010750, HP:0031956, HP:000082..."
2,Group_3,"[HP:0030863, HP:0002922, HP:0000234, HP:003195...","[ENSG00000198888, ENSG00000198899, ENSG0000019...","[HP:0030863, HP:0002922, HP:0000234, HP:003195..."
3,Group_4,"[HP:0007517, HP:0003731, HP:0031956, HP:000082...","[ENSG00000279566, ENSG00000281775, ENSG0000027...","[HP:0007517, HP:0003731, HP:0031956, HP:000082..."
4,Group_5,"[HP:0002922, HP:0000234, HP:0000151, HP:003195...","[ENSG00000198712, ENSG00000198840, ENSG0000021...","[HP:0002922, HP:0000234, HP:0000151, HP:003195..."
5,Group_6,"[HP:0002922, HP:0000234, HP:0000151, HP:000082...","[ENSG00000198888, ENSG00000198899, ENSG0000021...","[HP:0002922, HP:0000234, HP:0000151, HP:000082..."
6,Group_7,"[HP:0030500, HP:0002922, HP:0005274, HP:001075...","[ENSG00000198888, ENSG00000198899, ENSG0000019...","[HP:0030500, HP:0002922, HP:0005274, HP:001075..."
7,Group_8,"[HP:0000151, HP:0000822, HP:0001382, HP:000055...","[ENSG00000274746, ENSG00000231543, ENSG0000020...","[HP:0000151, HP:0000822, HP:0001382, HP:000055..."
8,Group_9,"[HP:0002922, HP:0005274, HP:0003731, HP:000015...","[ENSG00000280618, ENSG00000277421, ENSG0000027...","[HP:0002922, HP:0005274, HP:0003731, HP:000015..."
9,Group_10,"[HP:0003731, HP:0005274, HP:0001382, HP:000082...","[ENSG00000277101, ENSG00000275482, ENSG0000027...","[HP:0003731, HP:0005274, HP:0001382, HP:000082..."


In [39]:
df_new = df[['id', 'updated_phenotypes', 'all_candidate_genes']]
df_new = df_new.rename(columns={'updated_phenotypes': 'positive_phenotypes'})
df_new

,id,positive_phenotypes,all_candidate_genes
0,Group_1,"[HP:0009101, HP:0005274, HP:0010750, HP:000015...","[ENSG00000281919, ENSG00000273936, ENSG0000028..."
1,Group_2,"[HP:0002922, HP:0010750, HP:0031956, HP:000082...","[ENSG00000261992, ENSG00000277909, ENSG0000028..."
2,Group_3,"[HP:0030863, HP:0002922, HP:0000234, HP:003195...","[ENSG00000198888, ENSG00000198899, ENSG0000019..."
3,Group_4,"[HP:0007517, HP:0003731, HP:0031956, HP:000082...","[ENSG00000279566, ENSG00000281775, ENSG0000027..."
4,Group_5,"[HP:0002922, HP:0000234, HP:0000151, HP:003195...","[ENSG00000198712, ENSG00000198840, ENSG0000021..."
5,Group_6,"[HP:0002922, HP:0000234, HP:0000151, HP:000082...","[ENSG00000198888, ENSG00000198899, ENSG0000021..."
6,Group_7,"[HP:0030500, HP:0002922, HP:0005274, HP:001075...","[ENSG00000198888, ENSG00000198899, ENSG0000019..."
7,Group_8,"[HP:0000151, HP:0000822, HP:0001382, HP:000055...","[ENSG00000274746, ENSG00000231543, ENSG0000020..."
8,Group_9,"[HP:0002922, HP:0005274, HP:0003731, HP:000015...","[ENSG00000280618, ENSG00000277421, ENSG0000027..."
9,Group_10,"[HP:0003731, HP:0005274, HP:0001382, HP:000082...","[ENSG00000277101, ENSG00000275482, ENSG0000027..."


In [40]:
def row_to_json(row):
    json_dict = {
        'id': row['id'],
        'positive_phenotypes': row['positive_phenotypes'],
        'all_candidate_genes': row['all_candidate_genes']
    }
    return json.dumps(json_dict)

# Write the patient information to a JSONLines file
with open('/home/lucytian/data/shepherd_data/patients/simulated_patients/patients_updated.jsonl', 'w') as f:
    for _, row in df_new.iterrows():
        f.write(row_to_json(row) + '\n')